In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.read_csv("transformed_data/merged_data_1.csv", index_col="id")
cities = df['city'].unique().tolist()

In [3]:
df["room_type"].value_counts()

room_type
Entire home/apt    32648
Private room       18693
Shared room          366
Name: count, dtype: int64

In [4]:
df['num_bedrooms'].value_counts().sort_values(ascending=False)

num_bedrooms
1     36333
2      9290
0      4485
3      1477
4        96
5        10
9        10
6         2
10        2
8         2
Name: count, dtype: int64

In [5]:
# filter number of bedrooms by room_type = Entire home/apt and count them
df[df['room_type'] == 'Entire home/apt']['num_bedrooms'].value_counts()

num_bedrooms
1    18489
2     8650
0     4026
3     1410
4       67
5        4
6        2
Name: count, dtype: int64

In [6]:
df_copy = df[(df['room_type'] == 'Entire home/apt') & (df['num_bedrooms'] == 2)].copy()
df_copy

,price_total,room_type,is_shared_room,is_private_room,max_guests,is_superhost,is_multi_listing,is_business_listing,cleanliness_score,guest_satisfaction_score,...,longitude,latitude,city,day_type,district,state,country_code,country_name,Crime_Index,Safety_Index
id,,,,,,,,,,,,,,,,,,,,,
8,1001.804420,Entire home/apt,False,False,4.0,False,0,0,9.0,96.0,...,4.86459,52.40175,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
21,933.845757,Entire home/apt,False,False,4.0,False,0,0,10.0,96.0,...,4.89088,52.36422,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
28,1032.971668,Entire home/apt,False,False,4.0,False,1,0,9.0,96.0,...,4.92432,52.37676,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
32,516.017154,Entire home/apt,False,False,4.0,False,0,0,9.0,100.0,...,4.88881,52.36397,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
33,602.254353,Entire home/apt,False,False,3.0,False,0,0,10.0,100.0,...,4.86021,52.34502,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,231.634060,Entire home/apt,False,False,3.0,False,1,0,7.0,90.0,...,16.35842,48.19046,Vienna,weekend,Wien Stadt,Vienna,AT,Austria,28.4,71.6
1777,168.758619,Entire home/apt,False,False,4.0,False,0,1,8.0,79.0,...,16.33940,48.21299,Vienna,weekend,Wien Stadt,Vienna,AT,Austria,28.4,71.6
1783,486.408153,Entire home/apt,False,False,6.0,True,0,0,10.0,99.0,...,16.39024,48.20891,Vienna,weekend,Wien Stadt,Vienna,AT,Austria,28.4,71.6


In [7]:
cities

['Amsterdam',
 'Athens',
 'Barcelona',
 'Berlin',
 'Budapest',
 'Lisbon',
 'London',
 'Paris',
 'Rome',
 'Vienna']

In [24]:
#make it remove all this cities but each on line

# cities.remove(['Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris'])
cities.remove('Barcelona')
cities.remove('Berlin')
cities.remove('Budapest')
cities.remove('Lisbon')
cities.remove('London')
cities.remove('Paris')

In [25]:
cities

['Rome', 'Vienna']

In [8]:
100/2 < 50

False

In [26]:
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# --- ROBUST RATING/FAV EXTRACTOR ---
def get_details_and_fav(driver, wait):
    rating, reviews, is_fav = "N/A", "0", 0 

    try:
        # Locate the review anchor (link or button)
        try:
            review_link = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//a[contains(@href, 'reviews')] | //button[contains(., 'reviews')]")
            ))
        except:
            # Fallback for "New" listings
            try:
                h1_parent = driver.find_element(By.TAG_NAME, "h1").find_element(By.XPATH, "./..").text
                if "New" in h1_parent: return "New", "0", 0
            except: pass
            return rating, reviews, is_fav

        # Get text content (cleanly)
        link_text = review_link.get_attribute("textContent").strip()
        clean_link = " ".join(link_text.split())
        
        # Get parent text (for standard listings where rating is outside link)
        parent_text = review_link.find_element(By.XPATH, "./..").get_attribute("textContent").strip()
        clean_parent = " ".join(parent_text.split())

        # --- SCENARIO A: GUEST FAVORITE ---
        if "Guest favorite" in clean_link or "Guest favorite" in clean_parent:
            is_fav = 1
            # Rating logic for fav
            r_match = re.search(r"(\d\.\d{1,2})", clean_parent)
            if r_match: rating = r_match.group(1)
            # Review logic for fav
            rev_match = re.search(r"(\d+)\s*Review", clean_parent, re.IGNORECASE)
            if rev_match: reviews = rev_match.group(1)

        # --- SCENARIO B: STANDARD ---
        else:
            is_fav = 0
            if "New" in clean_parent:
                rating = "New"
            else:
                r_match = re.search(r"(\d\.\d{1,2})", clean_parent)
                if r_match: rating = r_match.group(1)

            # Try link first, then parent for reviews
            rev_match = re.search(r"(\d+)\s*review", clean_link, re.IGNORECASE)
            if not rev_match:
                rev_match = re.search(r"(\d+)\s*review", clean_parent, re.IGNORECASE)
            if rev_match: reviews = rev_match.group(1)

    except: pass
    return rating, reviews, is_fav
all_data = []
city_count = 0

for city in cities:
    city_count += 1
    
    # --- MAIN SETUP ---
    URL = f"https://www.airbnb.com/s/{city}/homes?refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&search_type=filter_change&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2026-02-01&monthly_length=3&monthly_end_date=2026-05-01&price_filter_input_type=2&channel=EXPLORE&price_filter_num_nights=5&min_bedrooms=2&selected_filter_order%5B%5D=min_bedrooms%3A2&selected_filter_order%5B%5D=room_types%3AEntire%20home%2Fapt&update_selected_filters=true&query={city}&search_mode=regular_search&room_types%5B%5D=Entire%20home%2Fapt"

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    target_count = 50  # Target number of items to scrape per city
    # Initialize a local counter
    items_scraped_this_city = 0 
    try:
        driver.get(URL)
        wait = WebDriverWait(driver, 8)

        while items_scraped_this_city < target_count:
            print(f"\n=== SCRAPING CITY {city_count}/{len(cities)}: {city} ===\n")
            print(f"--- Scraping Page (Collected: {len(all_data)}) ---")
            
            try:
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-testid='card-container']")))
            except: break

            listings = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='card-container']")
            listing_urls = []
            for l in listings:
                try:
                    url = l.find_element(By.TAG_NAME, "a").get_attribute("href")
                    listing_urls.append(url)
                except: continue

            main_window = driver.current_window_handle

            for url in listing_urls:
                if items_scraped_this_city >= target_count: break
                
                try:
                    driver.execute_script(f"window.open('{url}', '_blank');")
                    driver.switch_to.window(driver.window_handles[-1])
                    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
                    
                    # A. Name
                    try: name = driver.find_element(By.TAG_NAME, "h1").text
                    except: name = "No Title"

                    # B. Superhost
                    is_superhost = "No"
                    try:
                        driver.execute_script("window.scrollBy(0, 500);")
                        time.sleep(0.5) 
                        body_text = driver.find_element(By.TAG_NAME, "body").text
                        if "Superhost" in body_text: is_superhost = "Yes"
                    except: pass

                    # C. Price (Robust)
                    price = driver.find_element(By.XPATH, "//div[@data-plugin-in-point-id='BOOK_IT_SIDEBAR']/div/div/div/div[1]/div[1]/div/div/span/div[1]/div/span/div/button/span").text
                    
                    # D. Rating/Fav (Robust)
                    rating, reviews, is_fav = get_details_and_fav(driver, wait)

                    all_data.append({
                        "Name": name, "city": city, "Price": price, "Superhost": is_superhost,
                        "is_fav": is_fav, "Rating": rating, "Reviews": reviews, "URL": url
                    })

                    print(f"[{len(all_data)}] {price} |{city}| ★{rating} ({reviews}) | Fav: {is_fav} | SH: {is_superhost}|")
                    time.sleep(random.uniform(1.0, 2.0))

                    driver.close()
                    driver.switch_to.window(main_window)
                    
                    # Increment the counter for successfully scraped items
                    items_scraped_this_city += 1
                    
                except Exception:
                    if len(driver.window_handles) > 1:
                        driver.close()
                        driver.switch_to.window(main_window)
                    continue

            if items_scraped_this_city < target_count:
                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Next']")
                    driver.execute_script("arguments[0].click();", next_btn)
                    time.sleep(4)
                except: break

    finally:
        print(f"\nCompleted! Total scraped: {len(all_data)}")
        driver.quit()


=== SCRAPING CITY 1/2: Rome ===

--- Scraping Page (Collected: 0) ---
[1] 17,770 kr NOK |Rome| ★5.0 (0181) | Fav: 1 | SH: Yes|
[2] 18,841 kr NOK |Rome| ★4.82 (8277) | Fav: 1 | SH: Yes|

=== SCRAPING CITY 1/2: Rome ===

--- Scraping Page (Collected: 2) ---

=== SCRAPING CITY 1/2: Rome ===

--- Scraping Page (Collected: 2) ---
[3] 7,193 kr NOK |Rome| ★5.0 (027) | Fav: 1 | SH: Yes|
[4] 14,193 kr NOK |Rome| ★4.93 (93107) | Fav: 1 | SH: Yes|
[5] 8,396 kr NOK |Rome| ★5.0 (012) | Fav: 1 | SH: No|
[6] 8,092 kr NOK |Rome| ★4.93 (93748) | Fav: 1 | SH: Yes|
[7] 8,576 kr NOK |Rome| ★4.98 (9853) | Fav: 1 | SH: Yes|
[8] 13,738 kr NOK |Rome| ★4.99 (99186) | Fav: 1 | SH: Yes|

=== SCRAPING CITY 1/2: Rome ===

--- Scraping Page (Collected: 8) ---
[9] 10,995 kr NOK |Rome| ★4.96 (9656) | Fav: 1 | SH: Yes|
[10] 13,190 kr NOK |Rome| ★5.0 (06) | Fav: 1 | SH: No|
[11] 12,077 kr NOK |Rome| ★4.9 (9146) | Fav: 1 | SH: Yes|
[12] 9,060 kr NOK |Rome| ★4.95 (95117) | Fav: 1 | SH: Yes|
[13] 18,463 kr NOK |Rome| ★5.

In [27]:
df_scraped = pd.DataFrame(all_data)
df_scraped

,Name,city,Price,Superhost,is_fav,Rating,Reviews,URL
0,In the heart of Rome- opera design apartment,Rome,"17,770 kr NOK",Yes,1,5.0,0181,https://www.airbnb.com/rooms/94704622950296795...
1,Gleaming Loft Chapel Leopardi near Colosseum,Rome,"18,841 kr NOK",Yes,1,4.82,8277,https://www.airbnb.com/rooms/11099508025730023...
2,Silvery Apartment Aqua Paola close to Vatican,Rome,"7,193 kr NOK",Yes,1,5.0,027,https://www.airbnb.com/rooms/13955673150753485...
3,Majestic apartment close to Spanish Steps,Rome,"14,193 kr NOK",Yes,1,4.93,93107,https://www.airbnb.com/rooms/89471360392533334...
4,Rione Monti Apt 2 bedrooms 2 bathrooms,Rome,"8,396 kr NOK",No,1,5.0,012,https://www.airbnb.com/rooms/14650379287891431...
...,...,...,...,...,...,...,...,...
95,"Red Palm - 100m2 - central, stylish",Vienna,€ 857,Yes,1,4.9,930,https://www.airbnb.com/rooms/12650236356772771...
96,Cozy Apartment in Traditional Viennese Palais,Vienna,€ 817,No,1,4.82,82200,https://www.airbnb.com/rooms/22363944?search_m...
97,"Panoramic terrace view, loft3 next to city centre",Vienna,"€ 1,104",Yes,1,4.9,9203,https://www.airbnb.com/rooms/59764565621792904...
98,Big penthouse #12 with terrasse and view,Vienna,"€ 1,441",Yes,1,4.95,95275,https://www.airbnb.com/rooms/17493858?search_m...


In [28]:
df_scraped.to_csv("transformed_data/seif_scraped_data3.csv", index=False)

In [29]:
df_scraped["city"].value_counts()

city
Rome      50
Vienna    50
Name: count, dtype: int64